In [1]:
# Imports
import torch
import os 
import sys
from dask_jobqueue import SLURMCluster
from distributed import Client
from pathlib import Path

import do_simulation_causalklgp as file

In [2]:
# Args setup
ntrial = 50
n = 500
n_int = 100
niter = 1000
front_door = True
minimise =  False
calibrate = False
sample_split = True
add_base_kernel_BO = True

In [3]:
# Cluster creation
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="3:0:0",
    
    job_extra_directives = ["-p medium,fast,cpu"],
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [4]:
# Submitting jobs
futures = []
for seed in range(ntrial):
        f = client.submit(file.main,
                              seed,
                              n,n_int,
                              niter = niter,
                              front_door = front_door,
                              minimise = minimise,
                              calibrate = calibrate,
                              sample_split = sample_split,
                              add_base_kernel_BO = add_base_kernel_BO
                             )
        futures += [f]

In [5]:
futures

[<Future: pending, key: main-1c5e277288a3047886cf62f280d48f37>,
 <Future: pending, key: main-c1c06afa4a7a75d5c9584c4e7627655f>,
 <Future: pending, key: main-d8ec07a665b3f1093c4dd7f1e43d65a2>,
 <Future: pending, key: main-7e2b124f61ca24e6e41c6c0aff26ace4>,
 <Future: pending, key: main-5ada68ed01f054d1612fb32bd6f8d01e>,
 <Future: pending, key: main-c8508554c690b17ded3f68d9cf5cfcdd>,
 <Future: pending, key: main-a0ec0de3f08154a2c57ade7b2de19ecd>,
 <Future: pending, key: main-912d182fc31a1827b59e97cba42fda68>,
 <Future: pending, key: main-f38de47a4a9a52c2fa3729f0796eb16e>,
 <Future: pending, key: main-29d673639d63dad5cba33acd332b2aed>,
 <Future: pending, key: main-70f45b4286d40d0ef1ac01a464fa6b63>,
 <Future: pending, key: main-7b3a6bfa29ee857877f8150152bd1f0e>,
 <Future: pending, key: main-91af11034628fddc935d5857fa328df8>,
 <Future: pending, key: main-10110fd5ea638a69ab413cfe8fe469e9>,
 <Future: pending, key: main-1c6c72a8342f4002c653ab611c71eddd>,
 <Future: pending, key: main-e99a3f7c041

In [6]:
# Getting results
results = client.gather(futures)

In [7]:
# Closing client
client.close()
cluster.close()

In [8]:
# Saving results
torch.save(obj = results,
           f = "simulation_causalklgp_ntrial={0}_n={1}_frontdoor={2}_minimise={3}_addbase={4}.pt".format(ntrial,
                                                                                             n,
                                                                                             front_door,
                                                                                             minimise,
                                                                                             add_base_kernel_BO)
          )